In [1]:
import requests,os,time
def ratelimit():
    "A function that handles the rate of your calls."
    time.sleep(0.5) # sleep one second.

class Connector():
  def __init__(self,logfile,overwrite_log=False,connector_type='requests',session=False,path2selenium='',n_tries = 5,timeout=30):
    """This Class implements a method for reliable connection to the internet and monitoring. 
    It handles simple errors due to connection problems, and logs a range of information for basic quality assessments
    
    Keyword arguments:
    logfile -- path to the logfile
    overwrite_log -- bool, defining if logfile should be cleared (rarely the case). 
    connector_type -- use the 'requests' module or the 'selenium'. Will have different since the selenium webdriver does not have a similar response object when using the get method, and monitoring the behavior cannot be automated in the same way.
    session -- requests.session object. For defining custom headers and proxies.
    path2selenium -- str, sets the path to the geckodriver needed when using selenium.
    n_tries -- int, defines the number of retries the *get* method will try to avoid random connection errors.
    timeout -- int, seconds the get request will wait for the server to respond, again to avoid connection errors.
    """
    
    ## Initialization function defining parameters. 
    self.n_tries = n_tries # For avoiding triviel error e.g. connection errors, this defines how many times it will retry.
    self.timeout = timeout # Defining the maximum time to wait for a server to response.
    ## not implemented here, if you use selenium.
    if connector_type=='selenium':
      assert path2selenium!='', "You need to specify the path to you geckodriver if you want to use Selenium"
      from selenium import webdriver 
      ## HIN download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases

      assert os.path.isfile(path2selenium),'You need to insert a valid path2selenium the path to your geckodriver. You can download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases'
      self.browser = webdriver.Firefox(executable_path=path2selenium) # start the browser with a path to the geckodriver.

    self.connector_type = connector_type # set the connector_type
    
    if session: # set the custom session
      self.session = session
    else:
      self.session = requests.session()
    self.logfilename = logfile # set the logfile path
    ## define header for the logfile
    header = ['id','project','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
    if os.path.isfile(logfile):        
      if overwrite_log==True:
        self.log = open(logfile,'w')
        self.log.write(';'.join(header))
      else:
        self.log = open(logfile,'a')
    else:
      self.log = open(logfile,'w')
      self.log.write(';'.join(header))
    ## load log 
    with open(logfile,'r') as f: # open file
        
      l = f.read().split('\n') # read and split file by newlines.
      ## set id
      if len(l)<=1:
        self.id = 0
      else:
        self.id = int(l[-1][0])+1
            
  def get(self,url,project_name):
    """Method for connector reliably to the internet, with multiple tries and simple error handling, as well as default logging function.
    Input url and the project name for the log (i.e. is it part of mapping the domain, or is it the part of the final stage in the data collection).
    
    Keyword arguments:
    url -- str, url
    project_name -- str, Name used for analyzing the log. Use case could be the 'Mapping of domain','Meta_data_collection','main data collection'. 
    """
     
    project_name = project_name.replace(';','-') # make sure the default csv seperator is not in the project_name.
    if self.connector_type=='requests': # Determine connector method.
      for _ in range(self.n_tries): # for loop defining number of retries with the requests method.
        ratelimit()
        t = time.time()
        try: # error handling 
          response = self.session.get(url,timeout = self.timeout) # make get call

          err = '' # define python error variable as empty assumming success.
          success = True # define success variable
          redirect_url = response.url # log current url, after potential redirects 
          dt = t - time.time() # define delta-time waiting for the server and downloading content.
          size = len(response.text) # define variable for size of html content of the response.
          response_code = response.status_code # log status code.
          ## log...
          call_id = self.id # get current unique identifier for the call
          self.id+=1 # increment call id
          #['id','project_name','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row to be written in the log.
          self.log.write('\n'+';'.join(map(str,row))) # write log.
          self.log.flush()
          return response,call_id # return response and unique identifier.

        except Exception as e: # define error condition
          err = str(e) # python error
          response_code = '' # blank response code 
          success = False # call success = False
          size = 0 # content is empty.
          redirect_url = '' # redirect url empty 
          dt = t - time.time() # define delta t

          ## log...
          call_id = self.id # define unique identifier
          self.id+=1 # increment call_id

          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row
          self.log.write('\n'+';'.join(map(str,row))) # write row to log.
          self.log.flush()
    else:
      t = time.time()
      ratelimit()
      self.browser.get(url) # use selenium get method
      ## log
      call_id = self.id # define unique identifier for the call. 
      self.id+=1 # increment the call_id
      err = '' # blank error message
      success = '' # success blank
      redirect_url = self.browser.current_url # redirect url.
      dt = t - time.time() # get time for get method ... NOTE: not necessarily the complete load time.
      size = len(self.browser.page_source) # get size of content ... NOTE: not necessarily correct, since selenium works in the background, and could still be loading.
      response_code = '' # empty response code.
      row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row 
      self.log.write('\n'+';'.join(map(str,row))) # write row to log file.
      self.log.flush()
    # Using selenium it will not return a response object, instead you should call the browser object of the connector.
    ## connector.browser.page_source will give you the html.
      return None,call_id

In [2]:
#importing
import bs4 as bs
from urllib.request import urlopen as ureq
from bs4 import BeautifulSoup as soup
import pandas as pd
import urllib.request
import requests, re
from geopy.distance import great_circle
import uuid

<br>
<br>


In [3]:
df_urls = pd.read_csv("indvidual_urls.csv")
ratings_df = pd.DataFrame()


#empty lists
count = 0
loc_list = []
reviewCount_list = []
distance_list = []
unique_list = []
price_class_list = []
main_rating_list = []
ranking_list = []
price_class_value_list = []


<br>

In [4]:
#giant for loop
for url in df_urls["Restaurant_links"][:10]:
    
    
    #Sofies ratings loop
    count = count+1
    print(count)

    trip = ureq(url)
    trip_html = trip.read()
    trip.close()
    trip_soup = soup(trip_html, "lxml")
    
    
    test = trip_soup.findAll(True, {"class":["restaurants-detail-overview-cards-RatingsOverviewCard__ratingText--1P1Lq", "restaurants-detail-overview-cards-RatingsOverviewCard__ratingBubbles--1kQYC"]})
    name = trip_soup.findAll(True, {"class":["ui_header h1"]})
    name = str(name)
    name = name[26:]
    name = name.replace("</h1>]", '')
      
    elements = []
    for x in test:
        elements.append(str(x))
    
    keys = elements[0::2]
    values = elements[1::2]
    keys.append("Name")
    values.append(str(name))

    keys[:] = [s.replace('<span class="restaurants-detail-overview-cards-RatingsOverviewCard__ratingText--1P1Lq">', '') for s in keys]
    keys[:] = [s.replace('</span>', '') for s in keys]
    values[:] = [s.replace('<span class="restaurants-detail-overview-cards-RatingsOverviewCard__ratingBubbles--1kQYC"><span class="ui_bubble_rating bubble_', '') for s in values]
    values[:] = [s.replace('"></span></span>', '') for s in values]
    
    ratings_dict = {}
    for i in range(len(keys)):
        ratings_dict[keys[i]] = values[i]
    
    #append
    ratings_df = ratings_df.append([ratings_dict], ignore_index=True)    
    
    #Najas location loop
    p = re.compile(r'"coords":"(.*?)"')
    r = requests.get(url)
    coords = p.findall(r.text)[1]
    loc_list.append(coords)
    
    #Naja review count
    reviewCount = str(trip_soup.find(class_="reviewCount"))
    reviewCount = reviewCount.split(">")[1].split("<")[0]
    reviewCount_list.append(reviewCount)
    
    
#out of loop
ratings_df["Location"] = loc_list
ratings_df["Number of reviews"] = reviewCount_list

1
2


C:\Users\User\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


3


C:\Users\User\Anaconda3\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


4
5
6
7
8
9
10


In [5]:
for url in df_urls["Restaurant_links"][:10]:    
    url = ureq(url)
    url_html = url.read()
    url.close()
    url_soup = soup(url_html,'lxml')
    
    #Exstracting price_class_number $$$
    price_class_number = str(url_soup.find('div', class_="header_links"))
    price_class = re.sub('[^$-]', '', price_class_number)
    price_class_list.append(price_class)

    #Exstracting number of bubbles
    bubbles = str(url_soup.find(class_="restaurants-detail-overview-cards-RatingsOverviewCard__overallRating--nohTl"))    
    main_rating = re.sub('[^0-9,.]', '', bubbles) #stripping all other than the ranking numbers
    main_rating_list.append(main_rating)
    
    #Exstracting list_ranking
    list_ranking =str(url_soup.find(class_="restaurants-detail-overview-cards-RatingsOverviewCard__ranking--17CmN").find('span', class_=""))
    ranking = re.sub('[^0-9,]', '', list_ranking)
    ranking_list.append(ranking)
    
    #Exstracting price_class_value
    price_class_value = str(url_soup.find(class_="restaurants-detail-overview-cards-DetailsSectionOverviewCard__tagText--1OH6h"))
    price_class_value = re.sub('[^0-9,.]', '', price_class_value).split('.')[-1]
    price_class_value_list.append(price_class_value)
    
ratings_df["Price class"] = price_class_list
ratings_df["Main rating"] = main_rating_list
ratings_df["Ranking on list"] = ranking_list
ratings_df["Max price value"] = price_class_value_list

<br>

In [6]:
Kgs_Nytorv = '55.679977,12.5841893' #longitude and latitude for Kongens Nytorv

#calculating distance from nytorv to the coordinates in the list
def distance(x):
    Start = ratings_df["Location"][x]
    Stop = Kgs_Nytorv
    distance_list.append(great_circle(Start, Stop).meters)
    
for x in ratings_df.index:
    distance(x)
    
#appending to df 
ratings_df["Distance from Kgs. Nytorv"] = distance_list 

<br>

In [7]:
#creates a unique id 
for x in ratings_df.index:
    unique_list.append(uuid.uuid1())

ratings_df["Unique ID"] = unique_list

<br>

In [8]:
#sorting by name of column
ratings_df = ratings_df.reindex(sorted(ratings_df.columns), axis=1)
ratings_df

,Atmosphere,Distance from Kgs. Nytorv,Food,Location,Main rating,Max price value,Name,Number of reviews,Price class,Ranking on list,Service,Unique ID,Value
0,NaN,458.939814,45,"55.68388,12.58657",4.5,,Basso Kobenhavn,341 reviews,--$$-$$$---------,18,45,9fef85f6-c800-11e9-9188-b8ee65f7c9fe,45
1,40,477.463003,45,"55.684223,12.585324",4.5,,Che Fe,578 reviews,--$$-$$$---------,24,45,9fef85f7-c800-11e9-8ec8-b8ee65f7c9fe,45
2,NaN,2615.609175,45,"55.70349,12.585388",4.5,"16,,",Souls,647 reviews,--$$-$$$---------,31,45,9fefacfa-c800-11e9-8cbb-b8ee65f7c9fe,45
3,45,2735.600080,50,"55.7036,12.572",4.5,,Geranium,"1,053 reviews",--$$$$---,8,45,9fefacfb-c800-11e9-bbb3-b8ee65f7c9fe,45
4,NaN,389.814371,50,"55.678543,12.589863",4.5,,Trattoria l’Alba,176 reviews,--$$-$$$---------,23,45,9fefacfc-c800-11e9-9001-b8ee65f7c9fe,45
5,50,15868.985808,50,"55.813576,12.495031",5.0,"163951,794",Sollerod Kro,337 reviews,--$$$$------,4,50,9fefacfd-c800-11e9-95a0-b8ee65f7c9fe,45
6,NaN,139.552473,50,"55.681232,12.584203",4.5,,Restaurant leaven,309 reviews,--$$$$------,16,50,9fefacfe-c800-11e9-b389-b8ee65f7c9fe,45
7,45,561.698165,50,"55.67783,12.592299",4.5,,Studio - The Standard,453 reviews,--$$$$---,22,50,9fefacff-c800-11e9-86b1-b8ee65f7c9fe,45
8,NaN,1810.575627,45,"55.688663,12.559759",4.5,,Feed Bistro,548 reviews,--$$-$$$---,28,45,9fefad00-c800-11e9-bfac-b8ee65f7c9fe,45
9,NaN,1115.107516,45,"55.684696,12.568494",4.5,1633997,Trattoria SUD,306 reviews,--$$-$$$---------,17,50,9fefad01-c800-11e9-9550-b8ee65f7c9fe,45


<br>

In [9]:
#send to csv
#ratings_df.to_csv("SHORT output_deailed_ratings.csv")